# DX 704 Week 1 Project

This week's project will build a portfolio risk and return model, and make investing recommendations for hypothetical clients.
You will collect historical data, estimate returns and risks, construct efficient frontier portfolios, and sanity check the certainty of the maximum return portfolio.

The full project description and a template notebook are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-01


Feel free to use optimization tools or libraries (such as CVXOPT or scipy.optimize) to perform any calculations required for this mini project.

### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Data

Collect historical monthly price data for the last 24 months covering 6 different stocks.
The data should cover 24 consecutive months including the last month that ended before this week's material was released on Blackboard.
To be clear, if a month ends between the Blackboard release and submitting your project, you do not need to add that month.

The six different stocks must include AAPL, SPY and TSLA.
At least one of the remaining 3 tickers must start with the same letter as your last name (e.g. professor Considine could use COIN).
This is to encourage diversity in what stocks you analyze; if you discuss this project with classmates, please make sure that you pick different tickers to differentiate your work.
Do not pick stocks with fewer than 24 consecutive months of price data.

In [121]:
# YOUR CHANGES HERE
import datetime
from dateutil.relativedelta import relativedelta
import yfinance as yf
import pandas as pd

stocks = ['AAPL', 'SPY', 'TSLA', 'SPOT', 'GOOG', 'NVDA']

end_date = datetime.date(2025, 8, 31)
start_date = end_date - relativedelta(months=24)
print(f"Start Date: {start_date}")
print(f"End Date: {end_date}")

historical_prices = pd.DataFrame()

data = {}
for stock in stocks:
    df = yf.download(
        stock,
        start=start_date,
        end=end_date + datetime.timedelta(days=1),
        interval="1mo", 
        auto_adjust=False, 
        progress=False
    )

    print(f"\n{stock}: {len(df)} rows")
    print(df.head()) 

    try:
        historical_prices[stock] = df.xs('Adj Close', axis=1, level=0)
    except (KeyError, ValueError):
        print(f"Error for {stock}")



Start Date: 2023-08-31
End Date: 2025-08-31

AAPL: 24 rows
Price        Adj Close       Close        High         Low        Open  \
Ticker            AAPL        AAPL        AAPL        AAPL        AAPL   
Date                                                                     
2023-09-01  169.549286  171.210007  189.979996  167.619995  189.490005   
2023-10-01  169.113541  170.770004  182.339996  165.669998  171.220001   
2023-11-01  188.107498  189.949997  192.929993  170.119995  171.000000   
2023-12-01  190.913681  192.529999  199.619995  187.449997  190.330002   
2024-01-01  182.851898  184.399994  196.380005  180.169998  187.149994   

Price           Volume  
Ticker            AAPL  
Date                    
2023-09-01  1337823000  
2023-10-01  1172816900  
2023-11-01  1099651600  
2023-12-01  1063181200  
2024-01-01  1187490800  

SPY: 24 rows
Price        Adj Close       Close        High         Low        Open  \
Ticker             SPY         SPY         SPY         SPY  

Save the data as a TSV file named "historical_prices.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
The date should be the last trading day of the month, so it may not be the last day of the month.
For example, the last trading day of November 2024 was 2024-11-29.
The remaining columns should contain the adjusted closing prices of the corresponding stock tickers on that day.


In [122]:
# YOUR CHANGES HERE

historical_prices.reset_index(inplace=True)
historical_prices = historical_prices.drop(columns=['index'], errors='ignore')
historical_prices.rename(columns={'Date': 'date'}, inplace=True)
historical_prices.to_csv("historical_prices.tsv", sep='\t', index=False)
historical_prices

,date,AAPL,SPY,TSLA,SPOT,GOOG,NVDA
0,2023-09-01,169.549286,416.396912,250.220001,154.639999,131.070908,43.472248
1,2023-10-01,169.113541,408.794342,200.839996,164.759995,124.559601,40.758274
2,2023-11-01,188.107498,446.135223,240.080002,185.110001,133.128647,46.745087
3,2023-12-01,190.913681,464.619934,248.479996,187.910004,140.097229,49.495625
4,2024-01-01,182.851898,473.933411,187.289993,215.350006,140.962097,61.499634
5,2024-02-01,179.232559,498.666534,201.880005,256.410004,138.954025,79.076813
6,2024-03-01,170.257355,513.378845,175.789993,263.899994,151.360291,90.315811
7,2024-04-01,169.115555,494.210327,183.279999,280.440002,163.667130,86.367622
8,2024-05-01,190.879257,519.207397,178.080002,296.779999,172.932068,109.589378
9,2024-06-01,209.401917,535.796509,197.880005,313.790009,182.336151,123.490837


Submit "historical_prices.tsv" in Gradescope.

## Part 2: Calculate Historical Asset Returns

Calculate the historical asset returns based on the price data that you previously collected.

In [125]:
# YOUR CHANGES HERE

historical_prices.set_index("date", inplace=True)
historical_returns = historical_prices.pct_change().dropna()
print(historical_returns)


                AAPL       SPY      TSLA      SPOT      GOOG      NVDA
date                                                                  
2023-10-01 -0.002570 -0.018258 -0.197346  0.065442 -0.049678 -0.062430
2023-11-01  0.112315  0.091344  0.195379  0.123513  0.068795  0.146886
2023-12-01  0.014918  0.041433  0.034988  0.015126  0.052345  0.058841
2024-01-01 -0.042227  0.020045 -0.246257  0.146027  0.006173  0.242527
2024-02-01 -0.019794  0.052187  0.077901  0.190666 -0.014245  0.285809
2024-03-01 -0.050076  0.029503 -0.129235  0.029211  0.089283  0.142128
2024-04-01 -0.006706 -0.037338  0.042608  0.062675  0.081308 -0.043715
2024-05-01  0.128691  0.050580 -0.028372  0.058266  0.056608  0.268871
2024-06-01  0.097039  0.031951  0.111186  0.057315  0.054380  0.126850
2024-07-01  0.054411  0.015374  0.172781  0.096083 -0.054917 -0.052699
2024-08-01  0.031160  0.023365 -0.077390 -0.003082 -0.046434  0.020082
2024-09-01  0.018645  0.017883  0.221942  0.074807  0.012598  0.017341
2024-1

Save the data as a TSV file named "historical_returns.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
Each row should have the date at the end of the month and the corresponding *relative* price changes.
For example, if the previous price was \$100 and the new price is \$110, the return value should be 0.10.
There should only be 23 rows of data in this file, since they are computed as the differences of 24 prices.

In [126]:
# YOUR CHANGES HERE
historical_returns.reset_index(inplace=True)
historical_returns.to_csv("historical_returns.tsv", sep='\t', index=False)
historical_returns

,date,AAPL,SPY,TSLA,SPOT,GOOG,NVDA
0,2023-10-01,-0.002570,-0.018258,-0.197346,0.065442,-0.049678,-0.062430
1,2023-11-01,0.112315,0.091344,0.195379,0.123513,0.068795,0.146886
2,2023-12-01,0.014918,0.041433,0.034988,0.015126,0.052345,0.058841
3,2024-01-01,-0.042227,0.020045,-0.246257,0.146027,0.006173,0.242527
4,2024-02-01,-0.019794,0.052187,0.077901,0.190666,-0.014245,0.285809
5,2024-03-01,-0.050076,0.029503,-0.129235,0.029211,0.089283,0.142128
6,2024-04-01,-0.006706,-0.037338,0.042608,0.062675,0.081308,-0.043715
7,2024-05-01,0.128691,0.050580,-0.028372,0.058266,0.056608,0.268871
8,2024-06-01,0.097039,0.031951,0.111186,0.057315,0.054380,0.126850
9,2024-07-01,0.054411,0.015374,0.172781,0.096083,-0.054917,-0.052699


Submit "historical_returns.tsv" in Gradescope.

## Part 3: Estimate Returns

Estimate the expected returns for each asset using the previously calculated return data.
Just compute the average (mean) return for each asset over your data set; do not use other estimators that have been mentioned.
This will serve as your estimate of expected return for each asset.

In [127]:
# YOUR CHANGES HERE

estimated_returns = historical_returns.drop(columns='date').mean()
estimated_returns

AAPL    0.015376
SPY     0.019807
TSLA    0.025313
SPOT    0.071078
GOOG    0.023812
NVDA    0.068640
dtype: float64

Save the estimated returns in a TSV file named "estimated_returns.tsv" and include a header row with the column names "asset" and "estimated_return".

In [128]:
# YOUR CHANGES HERE
estimated_returns_df = estimated_returns.reset_index()
estimated_returns_df.columns = ['asset', 'estimated_return']
estimated_returns_df.to_csv("estimated_return.tsv", sep='\t', index=False)
estimated_returns_df


,asset,estimated_return
0,AAPL,0.015376
1,SPY,0.019807
2,TSLA,0.025313
3,SPOT,0.071078
4,GOOG,0.023812
5,NVDA,0.068640


Submit "estimated_returns.tsv" in Gradescope.

## Part 4: Estimate Risk

Estimate the covariance matrix for the asset returns to understand how the assets move together.

In [129]:
# YOUR CHANGES HERE

estimated_covariance = historical_returns.drop(columns='date').cov()
estimated_covariance

,AAPL,SPY,TSLA,SPOT,GOOG,NVDA
AAPL,0.003591,0.000785,0.003234,0.000150,0.000736,0.000927
SPY,0.000785,0.001259,0.002339,0.001500,0.000747,0.002809
TSLA,0.003234,0.002339,0.026703,0.003435,0.004309,-0.000386
SPOT,0.000150,0.001500,0.003435,0.009447,-0.000770,0.001990
GOOG,0.000736,0.000747,0.004309,-0.000770,0.004830,0.001703
NVDA,0.000927,0.002809,-0.000386,0.001990,0.001703,0.014522


Save the estimated covariances to a TSV file named "estimated_covariance.tsv".
The header row should have a blank column name followed by the names of the assets.
Each data row should start with the name of an asset for that row, and be followed by the individual covariances corresponding to that row and column's assets.
(This is the format of pandas's `to_csv` method with `sep="\t"` when used on a covariance matrix as computed in the examples.)

In [130]:
# YOUR CHANGES HERE

estimated_covariance.to_csv("estimated_covariance.tsv", sep="\t", index=True, header=True)
estimated_covariance

,AAPL,SPY,TSLA,SPOT,GOOG,NVDA
AAPL,0.003591,0.000785,0.003234,0.000150,0.000736,0.000927
SPY,0.000785,0.001259,0.002339,0.001500,0.000747,0.002809
TSLA,0.003234,0.002339,0.026703,0.003435,0.004309,-0.000386
SPOT,0.000150,0.001500,0.003435,0.009447,-0.000770,0.001990
GOOG,0.000736,0.000747,0.004309,-0.000770,0.004830,0.001703
NVDA,0.000927,0.002809,-0.000386,0.001990,0.001703,0.014522


Submit "estimated_covariance.tsv" in Gradescope.

## Part 5: Construct the Maximum Return Portfolio

Compute the maximum return portfolio based on your previously estimated risks and returns.

In [147]:
# YOUR CHANGES HERE

import numpy as np
import cvxpy as cp

n = len(estimated_returns)
x = cp.Variable(n)
assets = estimated_returns_df['asset'].values
mu = estimated_returns_df['estimated_return'].values

objective = cp.Maximize(mu.T @ x)
objective
constraints = [x >= 0, cp.sum(x) ==1]

prob = cp.Problem(objective, constraints)

estimated_return_maximum = prob.solve()
estimated_return_maximum

max_idx = np.argmax(mu)

maximum_allocations = np.zeros(n)
maximum_allocations[max_idx] = 1.0


Save the maximum return portfolio in a TSV file named "maximum_return.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [ ]:
# YOUR CHANGES HERE

max_return_df = pd.DataFrame({
    'asset': assets,
    'allocation': maximum_allocations
})
max_return_df.to_csv("maximum_return.tsv", sep="\t", index=False)
max_return_df

,asset,allocation
0,AAPL,0.0
1,SPY,0.0
2,TSLA,0.0
3,SPOT,1.0
4,GOOG,0.0
5,NVDA,0.0


Submit "maximum_return.tsv" in Gradescope.

## Part 6: Construct the Minimum Risk Portfolio

Compute the minimum return portfolio based on your previously estimated risks and returns.

In [139]:
# YOUR CHANGES HERE

x_minimum_risk = cp.Variable(n)
objective_minimum_risk = cp.Minimize(x_minimum_risk.T @ estimated_covariance.to_numpy() @ x_minimum_risk)
prob_minimum_risk = cp.Problem(objective_minimum_risk, [0 <= x_minimum_risk, cp.sum(x_minimum_risk) == 1])
covariance_minimum_risk = prob_minimum_risk.solve()
covariance_minimum_risk

minimum_allocations = x_minimum_risk.value

Save the minimum risk portfolio in a TSV file named "minimum_risk.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [140]:
# YOUR CHANGES HERE

min_return_df = pd.DataFrame({
    'asset': assets,
    'allocation': minimum_allocations
})
min_return_df.to_csv("minimum_risk.tsv", sep="\t", index=False)
min_return_df

,asset,allocation
0,AAPL,1.321812e-01
1,SPY,7.604356e-01
2,TSLA,1.183701e-19
3,SPOT,6.738851e-03
4,GOOG,1.006444e-01
5,NVDA,-3.354856e-19


Submit "minimum_risk.tsv" in Gradescope.

## Part 7: Build Efficient Frontier Portfolios

Compute 101 portfolios along the mean-variance efficient frontier with evenly spaced estimated returns.
The first portfolio should be the minimum risk portfolio from part 4, and the last portfolio should be the maximum return portfolio from part 3.
The estimated return of each portfolio should be higher than the previous by one percent of the difference between the first and last portfolios.
That is, the estimated return of the portfolios should be similar to `np.linspace(min_risk_return, max_return, 101)`.


In [153]:
# YOUR CHANGES HERE

assets = estimated_returns_df['asset'].values
mu = estimated_returns_df['estimated_return'].values
Sigma = estimated_covariance.loc[assets, assets].to_numpy()
n = len(mu)

r_min = mu @ minimum_allocations
r_max = mu @ maximum_allocations

target_returns = np.linspace(r_min, r_max, 101)

portfolios = []
risks = []

for r_target in target_returns:
    x = cp.Variable(n)
    objective = cp.Minimize(cp.quad_form(x, Sigma))
    constraints = [
        x >= 0,
        cp.sum(x) == 1,
        mu @ x == r_target
    ]
    prob = cp.Problem(objective, constraints)
    prob.solve()

    alloc = x.value
    portfolios.append(alloc)
    risk = np.sqrt(alloc.T @ Sigma @ alloc)
    risks.append(risk)

rows = []
for i, (ret, risk, alloc) in enumerate(zip(target_returns, risks, portfolios)):
    row = [i, ret, risk] + list(alloc)
    rows.append(row)

columns = ["index", "return", "risk"] + list(assets)

df_frontier = pd.DataFrame(rows, columns=columns)


Save the portfolios in a TSV file named "efficient_frontier.tsv".
The header row should have columns "index", "return", "risk", and all the asset tickers.
Each data row should have the portfolio index (0-100), the estimated return of the portfolio, the estimated standard deviation (not variance) of the portfolio, and all the asset allocations (which should sum to one).

In [154]:
# YOUR CHANGES HERE
df_frontier.to_csv("efficient_frontier.tsv", sep="\t", index=False)
df_frontier

,index,return,risk,AAPL,SPY,TSLA,SPOT,GOOG,NVDA
0,0,0.019970,0.033861,1.321812e-01,7.604356e-01,2.811949e-20,0.006739,1.006444e-01,-2.116140e-20
1,1,0.020481,0.033871,1.323464e-01,7.458859e-01,2.739803e-20,0.016348,1.054192e-01,-2.063721e-20
2,2,0.020992,0.033899,1.325116e-01,7.313363e-01,2.667823e-20,0.025958,1.101941e-01,-2.011654e-20
3,3,0.021503,0.033948,1.326767e-01,7.167867e-01,2.595470e-20,0.035568,1.149690e-01,-1.959316e-20
4,4,0.022015,0.034015,1.328419e-01,7.022371e-01,2.523374e-20,0.045177,1.197438e-01,-1.907105e-20
...,...,...,...,...,...,...,...,...,...
96,96,0.069033,0.079657,-6.757587e-23,-4.721580e-23,-2.192672e-22,0.630610,2.551469e-02,3.438755e-01
97,97,0.069544,0.080529,-7.226122e-23,-5.020946e-23,-2.209237e-22,0.636505,1.443449e-02,3.490604e-01
98,98,0.070055,0.081409,-7.309115e-23,-5.078609e-23,-2.199795e-22,0.642400,3.354291e-03,3.542453e-01
99,99,0.070566,0.084846,5.817319e-20,5.344937e-20,4.785029e-20,0.790361,4.910262e-20,2.096394e-01


Submit "efficient_frontier.tsv" in Gradescope.

## Part 8: Check Maximum Return Portfolio Stability

Check the stability of the maximum return portfolio by resampling the estimated risk/return model.

Repeat 1000 times -
1. Use `np.random.multivariate_normal` to generate 23 return samples using your previously estimated risks and returns.
2. Estimate the return of each asset using that resampled return history.
3. Check which asset had the highest return in those resampled estimates.

This procedure is a reduced and simplified version of the Michaud resampled efficient frontier procedure that takes uncertainty in the risk model into account.

In [143]:
# YOUR CHANGES HERE

from collections import Counter
n_simulations = 1000 

n_periods = historical_returns.shape[0]

mu = estimated_returns_df['estimated_return'].values
Sigma = estimated_covariance.loc[estimated_returns_df['asset'], estimated_returns_df['asset']].to_numpy()
assets = estimated_returns_df['asset'].values
n_assets = len(assets)

max_assets = []
for _ in range(n_simulations):
    simulated_returns = np.random.multivariate_normal(mu, Sigma, size=n_periods)
    simulated_mu = simulated_returns.mean(axis=0)
    max_idx = np.argmax(simulated_mu)
    max_assets.append(assets[max_idx])

counts = Counter(max_assets)
counts_full = {asset: counts.get(asset, 0) for asset in assets}

stability_df = pd.DataFrame({
    'asset': list(counts_full.keys()),
    'count': list(counts_full.values())

}).sort_values('count', ascending=False).reset_index(drop=True)

print(stability_df)


  asset  count
0  SPOT    496
1  NVDA    430
2  TSLA     72
3  GOOG      2
4   SPY      0
5  AAPL      0


Save a file "max_return_probabilities.tsv" with the distribution of highest return assets.
The header row should have columns "asset" and "probability".
There should be a data row for each asset and its sample probability of having the highest return based on those 1000 resampled estimates.


In [144]:
# YOUR CHANGES HERE

stability_df['probability'] = stability_df['count'] / n_simulations
stability_df = stability_df.drop(columns='count')
stability_df.to_csv("max_return_probabilities.tsv", sep="\t", index=False)
stability_df

,asset,probability
0,SPOT,0.496
1,NVDA,0.430
2,TSLA,0.072
3,GOOG,0.002
4,SPY,0.000
5,AAPL,0.000


Submit "max_return_probabilities.tsv" in Gradescope.

## Part 9: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgements.txt" in Gradescope.

## Part 10: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.